<a href="https://colab.research.google.com/github/vicentcamison/idal_ia3/blob/main/5%20Procesado%20del%20lenguaje%20natural/Sesion%202/NLP_06b_Extraccio%CC%81n_caracteri%CC%81sticas_BoW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extracción de características *Bag of Words*

Primero importamos todas las librerías necesarias

In [ ]:
import pandas as pd
import numpy as np
import re
import string
import spacy
import gensim

pd.options.display.max_colwidth = None


Creamos un pequeño cuerpo de textos de ejemplo *(CORPUS)*

In [ ]:
corpus = ['El cielo es azul y bonito',
          'Me encanta el cielo azul, pero no el cielo plomizo',
          'Bonito cielo hacía ese día',
          'Hoy he desayunado huevos con jamón y tostadas',
          'Juan odia las tostadas y los huevos con jamón',
          'las tostadas de jamón están muy buenas']

## Limpieza del texto
Definimos una función simple de limpieza y normalización del texto y la aplicamos a nuestro corpus.

In [ ]:
nlp = spacy.load("es_core_news_sm")
def normalizar_doc(doc):
    '''Función que normaliza un texto cogiendo sólo
    las palabras en minúsculas mayores de 3 caracteres'''
    # separamos en tokens
    tokens = nlp(doc)
    # filtramos stopwords
    filtered_tokens = [t.lower_ for t in tokens if
                       len(t.text)>3 and
                       not t.is_space and
                       not t.is_punct]
    # juntamos de nuevo en una cadena
    doc = ' '.join(filtered_tokens)
    return doc

In [ ]:
#probamos la función
normalizar_doc(corpus[0])

'cielo azul bonito'

In [ ]:
corpus[0]

'El cielo es azul y bonito'

In [ ]:
#aplicamos a todo el corpus
norm_corpus = [normalizar_doc(doc) for doc in corpus]
norm_corpus

['cielo azul bonito',
 'encanta cielo azul pero cielo plomizo',
 'bonito cielo hacía',
 'desayunado huevos jamón tostadas',
 'juan odia tostadas huevos jamón',
 'tostadas jamón están buenas']

In [ ]:
#alternativamente
list(map(normalizar_doc, corpus))

['cielo azul bonito',
 'encanta cielo azul pero cielo plomizo',
 'bonito cielo hacía',
 'desayunado huevos jamón tostadas',
 'juan odia tostadas huevos jamón',
 'tostadas jamón están buenas']

# Librería `scikit-learn`
Implementamos el modelo Bag-of-Word (BoW) con `scikit-learn`

Contamos la frecuencia de aparición de los términos en cada documento, usando un vocabulario común. 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
cv.fit(norm_corpus) #también funcionaría cv.fit(map(normalizar_doc, corpus))

CountVectorizer()

In [ ]:
type(cv)

sklearn.feature_extraction.text.CountVectorizer

El modelo genera un diccionario con todas las palabras del vocabulario y asigna un índice único a cada palabra:

In [ ]:
cv.get_feature_names()

['azul',
 'bonito',
 'buenas',
 'cielo',
 'desayunado',
 'encanta',
 'están',
 'hacía',
 'huevos',
 'jamón',
 'juan',
 'odia',
 'pero',
 'plomizo',
 'tostadas']

In [ ]:
cv.vocabulary_

{'cielo': 3,
 'azul': 0,
 'bonito': 1,
 'encanta': 5,
 'pero': 12,
 'plomizo': 13,
 'hacía': 7,
 'desayunado': 4,
 'huevos': 8,
 'jamón': 9,
 'tostadas': 14,
 'juan': 10,
 'odia': 11,
 'están': 6,
 'buenas': 2}

A partir del vocabulario aprendido, generamos el vector BoW de cada documento creando una matriz:

In [ ]:
cv_matrix = cv.transform(norm_corpus)
cv_matrix.shape

In [ ]:
#matriz sparse
cv_matrix

In [ ]:
#sólo guarda info de las celdas no vacías
print(cv_matrix)

In [ ]:
cv_matrix = cv_matrix.toarray()
cv_matrix

Cada término único es una característica de la matriz generada:

In [ ]:
# obtenemos palabras únicas en el corpus
vocab = cv.get_feature_names()
# mostramos vectores de características BoW del corpus
pd.DataFrame(cv_matrix, columns=vocab)

In [ ]:
#id de las palabras del vocabulario
cv.vocabulary_.get('cielo')

In [ ]:
#si una palabra no está en el vocabulario...
cv.vocabulary_.get('lluvia')

### Aplicando el modelo a nuevos documentos
Cuando calculamos el vector BoW de un texto nuevo con el modelo no hay que volver a ajustar el vocabulario, por lo que los términos nuevos no se tendrán en cuenta:

In [ ]:
nuevo_corpus = ['El Cielo amenaza lluvia', 'Pedro desayuna tostadas de jamón con tomate']
cv_matrix_nueva = cv.transform(map(normalizar_doc, nuevo_corpus))
cv_matrix_nueva

In [ ]:
pd.DataFrame(cv_matrix_nueva.toarray(), columns=vocab)

### Modelos N-grams
Considera como términos del vocabulario cada secuencia de N palabras consecutivas que aparece en el texto (*n-gramas*).  
Por ejemplo para los *bigrams* del corpus (N=2):

In [ ]:
bv = CountVectorizer(ngram_range=(2,2))
bv_matrix = bv.fit_transform(norm_corpus)

In [ ]:
bv.get_feature_names()

In [ ]:
len(bv.get_feature_names())

In [ ]:
bv_matrix

In [ ]:
bv_matrix = bv_matrix.toarray()
vocab_bigram = bv.get_feature_names()
pd.DataFrame(bv_matrix, columns=vocab_bigram)

In [ ]:
bv_matrix.shape

In [ ]:
bv.get_feature_names()

Se puede establecer el rango de n-grams a `(1,2)` para obtener el conjunto de unigramas y bigramas del corpus.  
Para limitar el número de términos en el vocabulario del modelo BoW se puede limitar a los términos que aparecen en un mínimo de documentos con el parámetro `min_df`

In [ ]:
bv = CountVectorizer(ngram_range=(1,2), min_df=2)
bv_matrix = bv.fit_transform(norm_corpus)

bv_matrix = bv_matrix.toarray()
vocab_bigram = bv.get_feature_names()
pd.DataFrame(bv_matrix, columns=vocab_bigram)

In [ ]:
bv_matrix.shape

In [ ]:
vocab_bigram

### Ejercicio 1
Aplica el modelo de BoW con bigramas al nuevo corpus de texto

In [ ]:
#completar

# Librería `Gensim`
Para trabajar con la librería `Gensim` es necesario transformar los documentos en una lista de tokens.

In [ ]:
def word_tokenize(text):
    return [token.text for token in nlp.make_doc(text)]

Convertimos nuestros texto de ejemplo en una lista de tokens y visualizamos el primer documento como ejemplo:

In [ ]:
tokenized_corpus = [word_tokenize(doc) for doc in corpus]
tokenized_corpus

In [ ]:
def normalizar_doc_tokenize(doc):
    '''Función que normaliza un texto cogiendo sólo
    las palabras en minúsculas mayores de 3 caracteres'''
    # separamos en tokens
    tokens = nlp(doc)
    # filtramos stopwords
    filtered_tokens = [t.lower_ for t in tokens if
                       len(t.text)>3 and
                       not t.is_space and
                       not t.is_punct]

    return filtered_tokens

In [ ]:
normalizar_doc_tokenize(corpus[0])

In [ ]:
tokenized_corpus = [normalizar_doc_tokenize(doc) for doc in corpus]
tokenized_corpus

## Modelo Bag of Words
Se pasará al modelo de Gensim como:

In [ ]:
from gensim.corpora import Dictionary

diccionario = Dictionary(tokenized_corpus)

In [ ]:
diccionario

El ID de cada palabra del diccionario se obtiene con:

In [ ]:
diccionario.token2id

La librería `gensim` crea la matriz BoW con otro formato. A cada palabra distinta del corpus se le asigna un ID único. Por cada documento se genera una lista de tuplas (ID, frecuencia) con la frecuencia de aparición de cada palabra:

In [ ]:
diccionario.doc2bow(tokenized_corpus[0])

In [ ]:
diccionario.token2id['plomizo'] #ID de cada término

In [ ]:
diccionario[5] #término correspondiente a una ID

In [ ]:
diccionario.id2token #diccionario de palabras para cada ID

In [ ]:
mapped_corpus = [diccionario.doc2bow(text)
                 for text in tokenized_corpus]

In [ ]:
mapped_corpus

In [ ]:
for (i, tf) in mapped_corpus[1]:
    print(f"{diccionario[i]}: {tf}")

In [ ]:
#frec. de documentos de cada token
diccionario.dfs

In [ ]:
for i in diccionario.dfs:
    print(f"{diccionario[i]}: {diccionario.dfs[i]}")

In [ ]:
#frec. aparición total de cada token
diccionario.cfs

### Ejercicio 2
Recorre el diccionario `cfs` mostrando el término correspondiente para cada ID y su frecuencia

In [ ]:
#completar

## Aplicación de los modelos a nuevos textos
Para aplicar un modelo BoW o TF-IDF a un nuevo documento hay que utilizar los modelos ya entrenados en `gensim` sobre el corpus original
### Modelo BoW

In [ ]:
tokenized_nuevo_corpus = [normalizar_doc_tokenize(doc) for doc in nuevo_corpus]

mapped_nuevo_corpus = [diccionario.doc2bow(text)
                 for text in tokenized_nuevo_corpus]

mapped_nuevo_corpus

In [ ]:
tokenized_nuevo_corpus

In [ ]:
for (i, tf) in mapped_nuevo_corpus[1]:
    print(f"{diccionario[i]}: {tf}")

In [ ]:
#Más pythonico con 'map'
list(map(diccionario.doc2bow, map(normalizar_doc_tokenize, nuevo_corpus)))

In [ ]:
#o mejor incluso
list(map(lambda x: diccionario.doc2bow(normalizar_doc_tokenize(x)), nuevo_corpus))

### Ejercicio 3
Define una función que devuelva el BoW de una lista de nuevos textos (pasada como lista de *strings*) usando el diccionario y la función de normalización creadas anteriormente.

In [ ]:
def bow(corpus, diccionario=diccionario, normalizacion=normalizar_doc_tokenize):
    """Genera la matriz BoW de la lista de texto en 'corpus'
    usando el diccionario y la función de normalización
    pasados como argumentos"""
    
    #COMPLETAR

In [ ]:
bow(nuevo_corpus)